Following tutorial described here: https://docs.prefect.io/tutorials/first-steps/

In [2]:
from prefect import flow

In [3]:
@flow
def my_favorite_function():
    print("What is your favorite number?")
    return 42

In [4]:
print(my_favorite_function())

02:40:41.707 | INFO    | prefect.engine - Created flow run 'quantum-buffalo' for flow 'my-favorite-function'
02:40:42.124 | INFO    | Flow run 'quantum-buffalo' - Finished in state Completed()


What is your favorite number?
42


### Tasks are called from flows

In [6]:
import requests
from prefect import flow, task

@task(name='task_call_api', description='Calls API')
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task(name='task_parse_response', description='Parses response')
def parse_fact(response):
    fact = response["fact"]
    print(fact)
    return fact

@flow(name='flow_data_extraction', description='Calls API and parses response')
def api_flow(url):
    fact_json = call_api(url)
    fact_text = parse_fact(fact_json)
    return fact_text

api_flow("https://catfact.ninja/fact")

00:13:02.084 | INFO    | prefect.engine - Created flow run 'impartial-pillbug' for flow 'flow_data_extraction'
00:13:02.593 | INFO    | Flow run 'impartial-pillbug' - Created task run 'task_call_api-ded10bed-0' for task 'task_call_api'
00:13:02.593 | INFO    | Flow run 'impartial-pillbug' - Executing 'task_call_api-ded10bed-0' immediately...
00:13:03.590 | INFO    | Task run 'task_call_api-ded10bed-0' - Finished in state Completed()
00:13:03.648 | INFO    | Flow run 'impartial-pillbug' - Created task run 'task_parse_response-6803447a-0' for task 'task_parse_response'
00:13:03.656 | INFO    | Flow run 'impartial-pillbug' - Executing 'task_parse_response-6803447a-0' immediately...


200


00:13:03.832 | INFO    | Task run 'task_parse_response-6803447a-0' - Finished in state Completed()
00:13:03.914 | INFO    | Flow run 'impartial-pillbug' - Finished in state Completed()


The biggest wildcat today is the Siberian Tiger. It can be more than 12 feet (3.6 m) long (about the size of a small car) and weigh up to 700 pounds (317 kg).


'The biggest wildcat today is the Siberian Tiger. It can be more than 12 feet (3.6 m) long (about the size of a small car) and weigh up to 700 pounds (317 kg).'

### Tasks and flows both can be retried
#### https://docs.prefect.io/tutorials/flow-task-config/#flow-and-task-retries

In [9]:
from prefect import flow, task

# this tasks runs 3 times before the flow fails
@task(retries=2, retry_delay_seconds=10)
def failure():
    print('running')
    raise ValueError("bad code")

@flow
def test_retries():
    return failure()

test_retries()

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\tasks.py:167: UserWarning: A task named 'failure' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\3521611229.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\flows.py:176: UserWarning: A flow named 'test-retries' and defined at 'C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3012\3521611229.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
00:28:11.174 | INFO    | prefect.engine - Created flow run 'cyber-gazelle' for flow 'test-retries'
00:28:11.766 | INFO    | Flow run 'cyber-gazelle' - Created task run 'failure-f8b6b21b-0' 

running


00:28:21.316 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\engine.py", line 1221, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\utilities\asyncutils.py", line 68, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_w

running


00:28:30.501 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\engine.py", line 1221, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\prefect\utilities\asyncutils.py", line 68, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\prefect-learning-oQ7bxwUN-py3.10\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_w

running


ValueError: bad code